In [ ]:
import numpy as np

## Simple example on Poission Equation. 


Objective: solve

$$
    \frac{d^2u(x)}{dx^2} = f(x)
$$

With boundary conditions $u(0) = u(1) = 0$. 

Let's pick a source term: $f(x) = \pi^2\sin(\pi x)$. This means that the equation has an exact solution and it is $u(x) =\sin(\pi x)$.

A Physics-Informed Neural Network is just a normal NN (e.g., MLP) trained to minimize a loss that enforces the physics — in this case, the Poisson equation.
We define a neural net $\hat{u}_{\theta}(x)$, and **instead of training it with labeled data, we train it by penalizing it when it violates the differential equation.**

To train a PINN, we need to define the loss function. The residual of the PINN is 

$$
    r(x) = \frac{d^2\hat{u}_{\theta}(x)}{dx^2} - f(x)
$$

**FUNDAMENTAL CONCEPTS**: 
* AUTOMATIC DIFFERENTIATION (AD).
    * Forward and Reverse AD 
    * Reverse AD vs Backpropagation
* DIFFERENTIABLE PROGRAMMING
    * For instance: differentiable PIC code. Why even bother to make the PIC code differentiable if it's already fast? 
        * Once it’s differentiable:
            * Optimize initial conditions to reach desired states
            * Use gradient-based inverse modeling: “Given a final state, what beam velocity vb caused it?”
            * Couple it to a **neural network controller** or PDE and train both
            * Use it in physics-informed learning frameworks (like PINNs) with real data
        * This moves your simulation from static modeling into trainable, learnable physical systems.



In [1]:
# Example of forward and reverse automatic differentiation 

import jax
import jax.numpy as jnp

def f(x):
    return x**2 + jnp.sin(x)

# Derivative using forward mode
df_dx_fwd = jax.jacfwd(f)
# Derivative using reverse mode
df_dx_rev = jax.grad(f)

x = 2.0
print(df_dx_fwd(x))  # Forward-mode derivative
print(df_dx_rev(x))  # Reverse-mode derivative


3.5838532
3.5838532
